In [ ]:
from config import MYPATHS
import functions as f
import threading

import master_list_to_option_chain as mo
mo.run_master_list_to_option_chain()

t = threading.Thread(target=f.prevent_sleep_with_win_search, args=(300,), daemon=True)
t.start()

# -----------------------------------------------

import asyncio
import nest_asyncio
import ast
import pandas as pd
from pathlib import Path
from datetime import datetime
import mmap
import struct
from collections import defaultdict
import os
import time
import pickle

nest_asyncio.apply()

# -----------------------------------------------

# Global variable to store current simulation date
current_simulation_date = datetime.now().strftime('%Y-%m-%d')
global_feed_manager = None

def on_file_start(filename):
    filename = filename
    
    # Extract date from filename and update tick saver
    import re
    date_pattern = r'(\d{4}-\d{2}-\d{2})'
    match = re.search(date_pattern, filename)
    
    if match:
        simulation_date = match.group(1)
        
        # Update global variable for tick saver
        global current_simulation_date, global_feed_manager
        old_date = current_simulation_date
        current_simulation_date = simulation_date
        
        # Reinitialize tick saver if date changed
        if old_date != simulation_date and global_feed_manager:
            if global_feed_manager.tick_saver:
                global_feed_manager.tick_saver.stop()
            global_feed_manager.tick_saver = UltraFastTickSaver(global_feed_manager.data_path, True)
    else:
        print("⚠️ Could not extract date from filename, using current date")
        current_simulation_date = datetime.now().strftime('%Y-%m-%d')

class SimulationDataWriter:
    def __init__(self, base_path, simulation_mode):
        if simulation_mode:
            return
        
        today_date = datetime.now().strftime('%Y-%m-%d')
        sim_path = Path(base_path) / "Raw_Simulation_data" / "Raw_data_bin"
        sim_path.mkdir(parents=True, exist_ok=True)
        
        self.file_path = sim_path / f"{today_date}.bin"
        self.file_handle = open(self.file_path, 'ab', buffering=0)
        self.buffer = []
        self.buffer_size = 1
        self.lock = threading.Lock()
        print(f"📝 Simulation data writer: {self.file_path}")
    
    def add_raw_tick(self, tick_dict):
        if hasattr(self, 'file_handle'):
            with self.lock:
                self.buffer.append(tick_dict)
                if len(self.buffer) >= self.buffer_size:
                    self._flush_buffer()
    
    def _flush_buffer(self):
        if not self.buffer:
            return
        batch_data = pickle.dumps(self.buffer)
        self.file_handle.write(struct.pack('I', len(batch_data)))
        self.file_handle.write(batch_data)
        self.buffer.clear()
    
    def stop(self):
        if hasattr(self, 'file_handle'):
            with self.lock:
                if self.buffer:
                    self._flush_buffer()
            self.file_handle.close()
            print(f"✅ Simulation data saved to: {self.file_path}")

class UltraFastTickSaver:
    RECORD_SIZE = 32
    INITIAL_FILE_SIZE = 1 * 1024 * 1024
    GROWTH_SIZE = 1 * 1024 * 1024
    
    def __init__(self, base_path="market_data", simulation_mode=False):
        if simulation_mode:
            # Use global simulation date updated by on_file_start
            folder_date = current_simulation_date
        else:
            # Use current date for live mode
            folder_date = datetime.now().strftime('%Y-%m-%d')
        
        folder_name = "Simulation_Tick_Data" if simulation_mode else "Live_Tick_Data"
        self.base_path = Path(base_path) / folder_name / f"market_data_{folder_date}"
        self.base_path.mkdir(parents=True, exist_ok=True)
        
        self.mmaps = {}
        self.files = {}
        self.positions = {}
        self.locks = defaultdict(threading.Lock)
            
    def _ensure_file_exists(self, security_id):
        if security_id in self.mmaps:
            return
        
        filepath = self.base_path / f"{security_id}.bin"
        
        if filepath.exists():
            # File exists - find actual data end, not just file size
            file_size = filepath.stat().st_size
            file_handle = open(filepath, 'r+b')
            mmap_obj = mmap.mmap(file_handle.fileno(), 0, access=mmap.ACCESS_WRITE)
            
            # Quick scan from end to find last record with data
            actual_position = 0
            for pos in range(file_size - self.RECORD_SIZE, -1, -self.RECORD_SIZE):
                if pos < 0:
                    break
                if mmap_obj[pos] != 0:  # Found last record with data
                    actual_position = pos + self.RECORD_SIZE
                    break
            
            self.positions[security_id] = actual_position
        else:
            # New file - create and start from beginning
            with open(filepath, 'wb') as f_file:
                f_file.write(b'\x00' * self.INITIAL_FILE_SIZE)
            file_handle = open(filepath, 'r+b')
            mmap_obj = mmap.mmap(file_handle.fileno(), 0, access=mmap.ACCESS_WRITE)
            self.positions[security_id] = 0
        
        self.files[security_id] = file_handle
        self.mmaps[security_id] = mmap_obj
    
    def _grow_file_if_needed(self, security_id):
        current_pos = self.positions[security_id]
        current_size = len(self.mmaps[security_id])
        
        if current_pos + (500 * self.RECORD_SIZE) >= current_size:
            # Close ONLY this specific file's mmap
            self.mmaps[security_id].close()
            
            # Grow ONLY this specific file
            self.files[security_id].seek(0, 2)  # Go to end of THIS file
            self.files[security_id].write(b'\x00' * self.GROWTH_SIZE)
            self.files[security_id].flush()
            
            # Recreate mmap ONLY for this specific file
            self.mmaps[security_id] = mmap.mmap(self.files[security_id].fileno(), 0, access=mmap.ACCESS_WRITE)
    
    def add_tick_instant(self, security_id, price, ltt, oi=0, volume=0):
        with self.locks[security_id]:
            self._ensure_file_exists(security_id)
            self._grow_file_if_needed(security_id)
            
            time_bytes = ltt.encode('utf-8')[:8].ljust(8, b'\x00')
            
            record = struct.pack('8sddd', time_bytes, float(price), float(oi), float(volume))
            
            pos = self.positions[security_id]
            self.mmaps[security_id][pos:pos + self.RECORD_SIZE] = record
            
            self.mmaps[security_id].flush()

            self.positions[security_id] += self.RECORD_SIZE
    
    def get_file_stats(self):
        stats = {}
        for security_id in self.positions:
            record_count = self.positions[security_id] // self.RECORD_SIZE
            file_size = len(self.mmaps[security_id]) if security_id in self.mmaps else 0
            stats[security_id] = {
                'records': record_count,
                'file_size_mb': file_size / (1024 * 1024),
                'position': self.positions[security_id]
            }
        return stats
    
    def stop(self):
        for security_id in list(self.mmaps.keys()):
            self.mmaps[security_id].close()
            self.files[security_id].close()
        self.mmaps.clear()
        self.files.clear()
        print("✅ All files closed")

class UltraFastMarketFeed:
    def __init__(self, dhan_context, instruments, max_instruments_per_conn=40, 
                 data_path="market_data", simulation_mode=False):
        self.dhan_context = dhan_context
        self.max_instruments_per_conn = min(max_instruments_per_conn, 50)
        self.running = False
        self.simulation_mode = simulation_mode
        self.data_path = data_path
        
        # Don't initialize tick_saver yet in simulation mode - wait for date
        if simulation_mode:
            self.tick_saver = None
        else:
            self.tick_saver = UltraFastTickSaver(data_path, simulation_mode)
            
        self.sim_writer = SimulationDataWriter(data_path, simulation_mode)
        
        self.instrument_batches = self._split_instruments(instruments)
        
        self.tick_count = 0
        self.start_time = time.time()
    
    def _split_instruments(self, instruments):
        batches = []
        for i in range(0, len(instruments), self.max_instruments_per_conn):
            batch = instruments[i:i + self.max_instruments_per_conn]
            batches.append(batch)
        return batches
    
    async def _handle_connection(self, instruments_batch, batch_id, total_batches):
        while self.running:
            try:
                mode_text = "🎮 SIMULATION" if self.simulation_mode else "📡 LIVE"
                
                f.printk(f"🔌 Batch {batch_id} starting...", not self.simulation_mode)
                
                feed = MarketFeed(self.dhan_context, instruments_batch, "v2")
                
                if self.simulation_mode:
                    feed.set_file_start_callback(on_file_start)
                    # Set simulation data path from notebook variable
                    feed.data_manager.set_simulation_data_path(simulation_data_path)
                
                await feed.connect()
                
                # Initialize tick_saver after first file loads in simulation mode
                if self.simulation_mode and self.tick_saver is None:
                    self.tick_saver = UltraFastTickSaver(self.data_path, self.simulation_mode)
                
                f.printk(f"✅ Batch {batch_id} connected", not self.simulation_mode)
                f.printk(f"✅ Simulator connected!", self.simulation_mode)
                
                while self.running:
                    try:
                        response = await feed.get_instrument_data()
                        if response and isinstance(response, dict):
                            print(response)
                            security_id = response.get('security_id')
                            ltp = response.get('LTP')
                            ltt = response.get('LTT')
                            
                            if security_id and ltp is not None and ltt:
                                if self.tick_saver:  # Ensure tick_saver exists
                                    self.tick_saver.add_tick_instant(security_id, ltp, ltt)
                                
                                if not self.simulation_mode:
                                    self.sim_writer.add_raw_tick(response)
                                
                                self.tick_count += 1
                                
                                if self.tick_count % 20000 == 0:
                                    elapsed = time.time() - self.start_time
                                    tps = self.tick_count / elapsed
                                    # print(f"📊 {self.tick_count} ticks, {tps:.0f} ticks/sec")
                        else:
                            if self.simulation_mode and response is None:
                                print("📊 Simulation data complete - stopping feed")
                                self.stop()
                                return
                            
                            if not self.simulation_mode:
                                current_time = datetime.now().strftime('%H:%M:%S')
                                if current_time > "15:30:00":
                                    print("📊 Market closed (15:30) - stopping feed")
                                    self.stop()
                                    return
                                
                    except Exception as e:
                        f.printk(f"❌ Data error in batch {batch_id}: {e}", not self.simulation_mode)
                        f.printk(f"❌ Data error [{mode_text}]: {e}", self.simulation_mode)
                        if not self.simulation_mode and total_batches == int(batch_id[-1:]):
                            Internet_Connection = f.is_internet_available()
                        await asyncio.sleep(0.001)
                        break
                        
            except Exception as e:
                f.printk(f"❌ Connection error in batch {batch_id}: {e}", not self.simulation_mode)
                f.printk(f"❌ Connection error [{mode_text}]: {e}", self.simulation_mode)
                await asyncio.sleep(1)
    
    async def start_all_connections(self):
        self.running = True
        self.start_time = time.time()
        
        tasks = []
        for i, batch in enumerate(self.instrument_batches):
            task = asyncio.create_task(
                self._handle_connection(batch, f"batch_{i}", len(tasks))
            )
            tasks.append(task)
                
        try:
            await asyncio.gather(*tasks)
        except KeyboardInterrupt:
            self.stop()
    
    def stop(self):
        print("🛑 Stopping ultra-fast feed...")
        self.running = False
        if self.tick_saver:
            self.tick_saver.stop()
        
        if hasattr(self, 'sim_writer'):
            self.sim_writer.stop()
        
        elapsed = time.time() - self.start_time
        tps = self.tick_count / elapsed if elapsed > 0 else 0
        print(f"📊 Final: {self.tick_count} ticks in {elapsed:.1f}s = {tps:.0f} ticks/sec")
    
    def get_stats(self):
        elapsed = time.time() - self.start_time if self.start_time else 0
        tps = self.tick_count / elapsed if elapsed > 0 else 0
        
        return {
            'total_ticks': self.tick_count,
            'ticks_per_second': tps,
            'elapsed_seconds': elapsed,
            'total_connections': len(self.instrument_batches),
            'file_stats': self.tick_saver.get_file_stats() if self.tick_saver else {}
        }

# ===============================================

MarketFeed_Simulation_or_live = "Live"      # "Live" "Simulation"

# ---------------------------------

if MarketFeed_Simulation_or_live == "Live":
    from dhanhq import dhanhq, DhanContext, MarketFeed

    today_date = datetime.now().strftime('%Y-%m-%d')
    file_path = MYPATHS['Option_Chain_with_Security_id'] + f"\\{today_date}.csv"

    option_chain_df = pd.read_csv(file_path)

    filtered_df = f.get_prices_under_1000(option_chain_df)
    instruments = f.create_instruments_from_df(filtered_df)

    print(f"📊 Instruments : {len(instruments)}, 🎮 Mode : LIVE")

# ---------------------------------

if MarketFeed_Simulation_or_live == "Simulation":
    from dhanhq import dhanhq, DhanContext
    from DhanHQ_simulator import MarketFeed

    # simulation_data_path = MYPATHS['Raw_data_bin'] + "\\2025-06-19.bin"   # Single file
    simulation_data_path = MYPATHS['Raw_data_bin']                          # Entire folder

    instruments = [("NSE_FNO", "999999", MarketFeed.Ticker)]

    print(f"🎮 Mode : SIMULATION")

# -----------------------------------------------

database_path = MYPATHS['data'] + "\\database.txt"
client_id = str(ast.literal_eval(f.get_line(database_path, 3).strip())['client_id'])
access_token = str(ast.literal_eval(f.get_line(database_path, 4).strip())['access_token'])

dhan_context = DhanContext(client_id, access_token)
dhan = dhanhq(dhan_context)

async def main():
    global global_feed_manager
    is_simulation = MarketFeed_Simulation_or_live == "Simulation"
    
    feed_manager = UltraFastMarketFeed(
        dhan_context, 
        instruments, 
        max_instruments_per_conn=40,
        data_path=MYPATHS['base'],
        simulation_mode=is_simulation
    )
    
    # Set global reference for callback access
    global_feed_manager = feed_manager
    
    try:
        await feed_manager.start_all_connections()
        
    except KeyboardInterrupt:
        print("🛑 Stopping feed...")
        feed_manager.stop()
    except Exception as e:
        print(f"❌ Feed error: {e}")
        feed_manager.stop()

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except KeyboardInterrupt:
        print("🛑 Feed stopped by user")
    except Exception as e:
        print(f"❌ Fatal error: {e}")
